In [25]:
# 使用numpy库来模拟向量、主存和向量寄存器组
import numpy as np

In [26]:
# 主存大小为4096，前1024为x[0:1023], 接下来1024为y[0:1023]
Mem = np.zeros(4096)
Mem[0:1024] = 1
Mem[1024:2048] = 2

# 向量寄存器组,Regs[0]为向量寄存器0，依此类推
Regs = np.zeros((64, 64))
# print(Regs)``

In [27]:
'''
addr:取得数的主存地址
dst_vector_num：目的寄存器（大小为64个元素）
'''
class LV:
    def __init__(self, addr, dst_vector_num):
        self.cur_cycle = 0
        self.addr = addr
        self.dst_vector_num = dst_vector_num

    def reset(self, addr, dst_vector_num):
        self.cur_cycle = 0
        self.addr = addr
        self.dst_vector_num = dst_vector_num
        
    def cycle(self):
        global Mem, Regs
        if(self.cur_cycle < 5):
            pass
        else:
            Regs[self.dst_vector_num][self.cur_cycle - 5] = Mem[self.addr]
            self.addr += 1
            
        if self.cur_cycle - 5 == 63:
            print("LV finished in %d cycles" % (self.cur_cycle))
            return False
        self.cur_cycle += 1
        return True

In [28]:
'''
addr:取得数的主存地址
dst_vector_num：目的寄存器（大小为64个元素）
'''
class SV:
    def __init__(self, addr, dst_vector_num):
        self.cur_cycle = 0
        self.addr = addr
        self.dst_vector_num = dst_vector_num

    def reset(self, addr, dst_vector_num):
        self.cur_cycle = 0
        self.addr = addr
        self.dst_vector_num = dst_vector_num
        
    def cycle(self):
        global Mem, Regs
        if(self.cur_cycle < 6):
            pass
        else:
            Mem[self.addr] = Regs[self.dst_vector_num][self.cur_cycle - 6]
            self.addr += 1
            
        if self.cur_cycle - 6 == 63:
            print("SV finished in %d cycles" % (self.cur_cycle))
            return False
        self.cur_cycle += 1
        return True


In [29]:
'''
向量数乘部件，给定一个标量和向量，将结果存在目的向量寄存器中
a:标量
src_vector_num: 源向量寄存器
dst_vector_num: 目的向量寄存器
'''
class MultSV:
    def __init__(self, a, src_vector_num, dst_vector_num):
        self.cur_cycle = 0
        self.a = a
        self.src_vector_num = src_vector_num
        self.dst_vector_num = dst_vector_num

    def reset(self, a, src_vector_num, dst_vector_num):
        self.cur_cycle = 0
        self.a = a
        self.src_vector_num = src_vector_num
        self.dst_vector_num = dst_vector_num

    '''
    模拟MultSV部件一个周期内的行为
    向量乘每个元素需要9拍，故前0-7周期不操作，后每个周期完成一个元素的乘法
    '''
    def cycle(self):
        global Regs
        if self.cur_cycle < 8:
            pass
        else:
            Regs[self.dst_vector_num][self.cur_cycle - 8] = self.a * Regs[self.src_vector_num][self.cur_cycle - 8]
            
        # 这条向量指令已经执行完毕
        if self.cur_cycle - 8 == 63:    
            # debug
            print("MulSV finished in %d cycles" %self.cur_cycle)
            return False
        self.cur_cycle += 1
        return True
        

In [30]:
"""
向量加部件，给定两个向量，将结果存在目的向量寄存器中
src_vector_num1：向量1
src_vector_num2：向量2
dst_vector_num：目的向量寄存器
"""
class AddV:
    def __init__(self,src_vector_num1,src_vector_num2,dst_vector_num):
        self.cur_cycle=0
        self.src_vector_num1=src_vector_num1
        self.src_vector_num2=src_vector_num2
        self.dst_vector_num=dst_vector_num

    def reset(self,src_vector_num1,src_vector_num2,dst_vector_num):
        self.cur_cycle=0
        self.src_vector_num1=src_vector_num1
        self.src_vector_num2=src_vector_num2
        self.dst_vector_num=dst_vector_num
    
    """
    模拟AddV部件一个周期的行为
    向量加每个元素需要5拍，0到4周期不操作
    """
    def cycle(self):
        global Regs
        if self.cur_cycle < 4:
            pass
        else:
            Regs[self.dst_vector_num][self.cur_cycle-4]=Regs[self.src_vector_num1][self.cur_cycle-4]+Regs[self.src_vector_num2][self.cur_cycle-4]
        
        # 这条向量指令已经执行完毕
        if self.cur_cycle - 4 == 63:    
            # debug
            print("MulSV finished in %d cycles" %self.cur_cycle)
            return False
        self.cur_cycle += 1
        return True

In [31]:
# 输入a
a = np.float64(input("input a const a"))
# 创建各个部件，模拟硬件
sv_unit = SV(1024, 4)
lv_unit = LV(0, 1)
multsv_unit = MultSV(a, 1, 2)
addv_unit = AddV(2, 3, 4)

# 分组进行，共需要1024//64个组
times = 1024//64
# 总的周期数
total_cycles = 0

In [32]:
# 每次先刷新各组件内的成员值
for i in range(times):
    sv_unit.reset(1024 + times*64, 4)
    lv_unit.reset(0 + times*64, 1)
    multsv_unit.reset(a, 1, 2)
    addv_unit.reset(2, 3, 4)

    # 首先LV部件加载主存中的数据到寄存器
    while True:
        lv_result = lv_unit.cycle()
        total_cycles += 1
        if lv_result == False:
            break

    # 刷新lv部
    lv_unit.reset(1024 + times * 64, 3)
    # 实际上是并行，这里没有用多线程
    while True:
        lv_result = lv_unit.cycle()
        multsv_result = multsv_unit.cycle()

        # 链接，上面两个部件的结果做加法
        if multsv_unit.cur_cycle < 9+1 or lv_unit.cur_cycle < 6+1:
            pass
        else:
            addv_result = addv_unit.cycle()
        # 链接，加法的结果直接存
        if addv_unit.cur_cycle < 5+1:
            pass
        else:
            sv_result = sv_unit.cycle()
        total_cycles += 1
        if lv_result == False and multsv_result == False and addv_result== False and sv_result == False:
            break
        
print("total_cycles = %d" %total_cycles)
print(Mem)

LV finished in 68 cycles
LV finished in 68 cycles
LV finished in 68 cycles
LV finished in 68 cycles
LV finished in 68 cycles
MulSV finished in 71 cycles
LV finished in 68 cycles
MulSV finished in 71 cycles
LV finished in 68 cycles
MulSV finished in 71 cycles
LV finished in 68 cycles
MulSV finished in 71 cycles
LV finished in 68 cycles
MulSV finished in 71 cycles
LV finished in 68 cycles
MulSV finished in 71 cycles
MulSV finished in 67 cycles
LV finished in 68 cycles
MulSV finished in 71 cycles
MulSV finished in 67 cycles
LV finished in 68 cycles
MulSV finished in 71 cycles
MulSV finished in 67 cycles
LV finished in 68 cycles
MulSV finished in 71 cycles
MulSV finished in 67 cycles
LV finished in 68 cycles
MulSV finished in 71 cycles
MulSV finished in 67 cycles
LV finished in 68 cycles
MulSV finished in 71 cycles
MulSV finished in 67 cycles
LV finished in 68 cycles
MulSV finished in 71 cycles
MulSV finished in 67 cycles
LV finished in 68 cycles
MulSV finished in 71 cycles
MulSV finished 